# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [18]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

# Solution for Q1

In [20]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value
    print(sqrt_value)

print("\nSum of the outputs of the generator for limit = 5:", sum)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979

Sum of the outputs of the generator for limit = 5: 8.382332347441762


# Solution for Q2

In [21]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value
    print(sqrt_value)

print("\nThe 13th number yielded by the generator:", sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989

The 13th number yielded by the generator: 3.605551275463989


# Solution for Q3

In [6]:
import dlt
import duckdb

In [10]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")


# connect to duckdb
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data
print("\n\n\n people table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

# Sum of ages of all people from first generator
print("\n\n\n Sum of ages of all people from first generator:")

people = conn.sql("SELECT sum(age) FROM people").df()
display(people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708970417.0601387,gAAGV7Q8Gmq6Nw
1,2,Person_2,27,City_A,1708970417.0601387,nIivoVMDj08VAw
2,3,Person_3,28,City_A,1708970417.0601387,7Ez/qw2vXqRdKw
3,4,Person_4,29,City_A,1708970417.0601387,SwFT0fIMfjLpBg
4,5,Person_5,30,City_A,1708970417.0601387,LfIHobr+lumIrQ





 Sum of ages of all people from first generator:


,sum(age)
0,140.0


In [12]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}
    
# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

# and the data
print("\n\n\n people table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

# Sum of ages of all people
print("\n\n\n Sum of ages of all people from both generators:")

people = conn.sql("SELECT sum(age) FROM people").df()
display(people)




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708970417.0601387,gAAGV7Q8Gmq6Nw,None
1,2,Person_2,27,City_A,1708970417.0601387,nIivoVMDj08VAw,None
2,3,Person_3,28,City_A,1708970417.0601387,7Ez/qw2vXqRdKw,None
3,4,Person_4,29,City_A,1708970417.0601387,SwFT0fIMfjLpBg,None
4,5,Person_5,30,City_A,1708970417.0601387,LfIHobr+lumIrQ,None
5,3,Person_3,33,City_B,1708970534.147054,Q2ftAi1jaQ0k9g,Job_3
6,4,Person_4,34,City_B,1708970534.147054,ev3Oz21UB05jnA,Job_4
7,5,Person_5,35,City_B,1708970534.147054,UGAiy3IM57ArGA,Job_5
8,6,Person_6,36,City_B,1708970534.147054,QW0vPrzxRE/tXg,Job_6
9,7,Person_7,37,City_B,1708970534.147054,boV+pyGebOcDxw,Job_7





 Sum of ages of all people from both generators:


,sum(age)
0,353.0


# Solution for Q4

In [14]:
# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="merged_people",
										write_disposition="replace",
                              primary_key='id')

people = conn.sql("SELECT * FROM merged_people").df()
display(people)

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708970787.1505628,vHOEoOQmlqhw3w
1,2,Person_2,27,City_A,1708970787.1505628,HroI0kYM0tmwqg
2,3,Person_3,28,City_A,1708970787.1505628,3Znyqwgvo82+uQ
3,4,Person_4,29,City_A,1708970787.1505628,w1PIAdbWaPxsbA
4,5,Person_5,30,City_A,1708970787.1505628,Tj5o+79FvaaBbQ


In [16]:
# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(),
										table_name="merged_people",
										write_disposition="merge",
                              primary_key='id')

people = conn.sql("SELECT * FROM merged_people").df()
display(people)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708970787.1505628,vHOEoOQmlqhw3w,None
1,2,Person_2,27,City_A,1708970787.1505628,HroI0kYM0tmwqg,None
2,5,Person_5,35,City_B,1708970815.8104696,bdDT+EtZmvIM3g,Job_5
3,4,Person_4,34,City_B,1708970815.8104696,U0Ei4Njq1tzXdA,Job_4
4,8,Person_8,38,City_B,1708970815.8104696,DWXXdoa9Ru9vsg,Job_8
5,7,Person_7,37,City_B,1708970815.8104696,sjW+jqeK1HeBRQ,Job_7
6,3,Person_3,33,City_B,1708970815.8104696,rqH96WASibDNYQ,Job_3
7,6,Person_6,36,City_B,1708970815.8104696,5tmTemE3Lhjyyw,Job_6


In [17]:
people = conn.sql("SELECT sum(age) FROM merged_people").df()
display(people)

,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX